<a href="https://colab.research.google.com/github/aleksandra1206/movies_recommender_system/blob/main/RecommenderSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=ad35b6133e61f743d7b7c5e94ee567e1710fc710cfffd391af0a8100f07228ff
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/Domaci1'

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MovieSuggestions').config("spark.executor.instances", "4").config("spark.executor.cores", "8").config("spark.executor.memory", "12g").getOrCreate()

Mounted at /content/drive


In [ ]:
import os
os.chdir(r"/content/drive/MyDrive/Domaci1")

from pyspark.sql import SparkSession
from pyspark.sql import functions as func

In [ ]:
movies = spark.read.csv(os.path.abspath("u.item"), inferSchema= True, header = False, sep = '|')
ratings = spark.read.csv(os.path.abspath("u.data"), inferSchema= True, header = False, sep = '\t')
genres = spark.read.csv(os.path.abspath("u.genre"), inferSchema= True, header = False, sep = '|')
occupations = spark.read.csv(os.path.abspath("u.occupation"), inferSchema= True, header = False)
users = spark.read.csv(os.path.abspath("u.user"), inferSchema= True, header = False, sep = '|')

movies.show()
ratings.show()
genres.show()
occupations.show()
users.show()

+---+--------------------+-----------+----+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|_c0|                 _c1|        _c2| _c3|                 _c4|_c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|
+---+--------------------+-----------+----+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|  1|    Toy Story (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  0|  0|  1|  1|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|
|  2|    GoldenEye (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  1|  1|  0|  0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   1|   0|   0|
|  3|   Four Rooms (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  0|  0|  0|  0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   1|   0|   0|
|  4|   Get Shorty (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  

# **Ucitavanje podataka**

In [ ]:
movie_columns = ['Id' , 'Title' , 'Release date' , 'Video release date' ,
              'IMDb URL' , 'Unknown' , 'Action' , 'Adventure' , 'Animation' ,
              "Children's" , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western']
movies = movies.toDF(*movie_columns)
print("Prikaz filmova i njihovih atributa:")
movies.show()

ratings_columns = ['User_id' , 'Movie_id' , 'Rating' , 'Timestamp' ]
ratings = ratings.toDF(*ratings_columns)
print("Prikaz rejtinga:")
ratings.show()

genre_columns = ['Name' , 'Id']
genres = genres.toDF(*genre_columns)
print("Prikaz žanrova:")
genres.show()

occupation_columns = ['Occupation']
occupations = occupations.toDF(*occupation_columns)
print("Prikaz zanimanja:")
occupations.show()

user_columns = ['Id', 'Age', 'Gender', 'Occupation', 'Zip code']
users = users.toDF(*user_columns)
print("Prikaz korisnika:")
users.show()

Prikaz filmova i njihovih atributa:
+---+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
| Id|               Title|Release date|Video release date|            IMDb URL|Unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+---+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|  1|    Toy Story (1995)| 01-Jan-1995|              NULL|http://us.imdb.co...|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|  2|    GoldenEye (1995)| 0

# **1. DEO: NAJPOPULARNIJI FILMOVI**

In [ ]:
print("NAJPOPULARNIJI FILMOVI")
from pyspark.sql.functions import col, avg, count, max, expr

# 1. Najpopularniji filmovi - 50 najpopularnijih filmova po broju pregleda
popular_movies = ratings.groupBy("Movie_id").agg(count("Rating").alias("Broj_pregleda"), avg("Rating").alias("Prosecna_ocena"))
popular_movies = popular_movies.join(movies, popular_movies.Movie_id == movies.Id).orderBy(col("Broj_pregleda").desc())
print("1. Prikaz 50 najpopularnijih filmova:")
popular_movies.show(50)

NAJPOPULARNIJI FILMOVI
1. Prikaz 50 najpopularnijih filmova:
+--------+-------------+------------------+---+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|Movie_id|Broj_pregleda|    Prosecna_ocena| Id|               Title|Release date|Video release date|            IMDb URL|Unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+-------------+------------------+---+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|      50|          583|4.3584905660377355| 50|    Star Wars (1977)| 01-Jan-1977|              NULL|http://us.i

**10 FILMOVA SA NAJVISE I 10 FILMOVA SA NAJMANJE PREGLEDA**



In [ ]:
# 2. Prikaz 10 filmova sa najviše i 10 filmova sa najmanje pregleda
top_10_most_viewed = popular_movies.orderBy(col("Broj_pregleda").desc()).limit(10)
top_10_least_viewed = popular_movies.orderBy(col("Broj_pregleda").asc()).limit(10)
print("2. Prikaz 10 filmova sa najviše i 10 filmova sa najmanje pregleda")
print("10 filmova sa najvise pregleda:")
top_10_most_viewed.show()
print("10 filmova sa najmanje pregleda:")
top_10_least_viewed.show()

2. Prikaz 10 filmova sa najviše i 10 filmova sa najmanje pregleda
10 filmova sa najvise pregleda:
+--------------------+-------------+------------------+
|               Title|Broj_pregleda|    Prosecna_ocena|
+--------------------+-------------+------------------+
|    Star Wars (1977)|          583|4.3584905660377355|
|      Contact (1997)|          509|3.8035363457760316|
|        Fargo (1996)|          508| 4.155511811023622|
|Return of the Jed...|          507| 4.007889546351085|
|    Liar Liar (1997)|          485| 3.156701030927835|
|English Patient, ...|          481| 3.656964656964657|
|       Scream (1996)|          478|3.4414225941422596|
|    Toy Story (1995)|          452|3.8783185840707963|
|Air Force One (1997)|          431|3.6310904872389793|
|Independence Day ...|          429| 3.438228438228438|
+--------------------+-------------+------------------+

10 filmova sa najmanje pregleda:
+--------------------+-------------+--------------+
|               Title|Broj_pregl

**IZRACUNAVANJE SKORA ZA SVAKI FILM I PRIKAZ TOP 5 FILMOVA PO SKORU**

In [ ]:
# 3. Izračunavanje "skora" za svaki film i prikaz top 5 filmova po "skoru"
max_pregledi = popular_movies.agg(max("Broj_pregleda")).first()[0]
max_ocena = popular_movies.agg(max("Prosecna_ocena")).first()[0]

skor_filma = popular_movies.withColumn("Skor", expr(f"(Broj_pregleda * Prosecna_ocena) / ({max_pregledi} * {max_ocena})"))
print("3. Izračunavanje skora za svaki film i prikaz top 5 filmova po skoru:")
skor_filma.orderBy(col("Skor").desc()).show(5)

3. Izračunavanje skora za svaki film i prikaz top 5 filmova po skoru:
+--------------------+-------------+------------------+------------------+
|               Title|Broj_pregleda|    Prosecna_ocena|              Skor|
+--------------------+-------------+------------------+------------------+
|    Star Wars (1977)|          583|4.3584905660377355|0.8716981132075472|
|        Fargo (1996)|          508| 4.155511811023622|0.7241852487135506|
|Return of the Jed...|          507| 4.007889546351085|0.6970840480274443|
|      Contact (1997)|          509|3.8035363457760316|0.6641509433962264|
|Raiders of the Lo...|          420| 4.252380952380952|0.6126929674099485|
+--------------------+-------------+------------------+------------------+
only showing top 5 rows



**5 NAJPOPULARNIJIH FILMOVA PO POLU**

In [ ]:
# 4. Prikaz 5 najpopularnijih filmova po polu
popular_movies_gender = ratings.join(users, users.Id==ratings.User_id).groupBy("Gender", "Movie_id").agg(count("Rating").alias("Broj_pregleda"), avg("Rating").alias("Prosecna_ocena"))
popular_movies_gender = popular_movies_gender.join(movies, popular_movies_gender.Movie_id == movies.Id).select("Gender", "Title", "Broj_pregleda", "Prosecna_ocena").orderBy(col("Broj_pregleda").desc())
print("4. Prikaz 5 najpopularnijih filmova po polu")
print("5 najpopularnijih filmova za muski pol:")
popular_movies_gender.filter(col("Gender") == "M").show(5)  # Najpopularniji filmovi kod muškaraca
print("5 najpopularnijih filmova za zenski pol:")
popular_movies_gender.filter(col("Gender") == "F").show(5)  # Najpopularniji filmovi kod žena

4. Prikaz 5 najpopularnijih filmova po polu
5 najpopularnijih filmova za muski pol:
+------+--------------------+-------------+------------------+
|Gender|               Title|Broj_pregleda|    Prosecna_ocena|
+------+--------------------+-------------+------------------+
|     M|    Star Wars (1977)|          432| 4.398148148148148|
|     M|        Fargo (1996)|          383| 4.201044386422977|
|     M|Return of the Jed...|          383| 4.007832898172324|
|     M|      Contact (1997)|          372| 3.846774193548387|
|     M|    Liar Liar (1997)|          344|3.1802325581395348|
+------+--------------------+-------------+------------------+
only showing top 5 rows

5 najpopularnijih filmova za zenski pol:
+------+--------------------+-------------+------------------+
|Gender|               Title|Broj_pregleda|    Prosecna_ocena|
+------+--------------------+-------------+------------------+
|     F|English Patient, ...|          152|3.8092105263157894|
|     F|    Star Wars (1977)|  

**3 NAJPOPULARNIJA FILMA PO POLU I ZANIMANJU KORISNIKA**

In [ ]:
# 5. Prikaz 3 najpopularnija filma po polu i zanimanju korisnika
from pyspark.sql import Window
from pyspark.sql.functions import col, count, avg, row_number

# Grupisanje po Gender, Occupation, Movie_id i izračunavanje broja pregleda i prosečne ocene
popular_movies_gender_occupation = ratings.join(users, users.Id == ratings.User_id) \
    .groupBy("Gender", "Occupation", "Movie_id") \
    .agg(count("Rating").alias("Broj_pregleda"), avg("Rating").alias("Prosecna_ocena"))

# Pridruživanje naziva filmova
popular_movies_gender_occupation = popular_movies_gender_occupation.join(
    movies, popular_movies_gender_occupation.Movie_id == movies.Id, "inner"
)

# Definisanje prozora particionisanog po Gender i Occupation, a sortiranog po Broj_pregleda
windowSpec = Window.partitionBy("Gender", "Occupation").orderBy(col("Broj_pregleda").desc())

# Dodavanje row_number() kolone da bismo rangirali filmove unutar svake particije
ranked_movies = popular_movies_gender_occupation.withColumn(
    "Rank", row_number().over(windowSpec)
)

# Filtriranje rezultata da uzmemo samo prva tri filma po polu i zanimanju
top_3_movies_per_gender_occupation = ranked_movies.filter(col("Rank") <= 3)

print("5. Prikaz 3 najpopularnija filma po polu i zanimanju korisnika:")
top_3_movies_per_gender_occupation.select(
    "Gender", "Occupation", "Title", "Broj_pregleda", "Prosecna_ocena"
).orderBy("Gender", "Occupation", "Rank")

from pyspark.sql.functions import col

# Dobijanje jedinstvenih vrednosti za Gender i Occupation
gender_list = [row.Gender for row in users.select("Gender").distinct().collect()]
occupation_list = [row.Occupation for row in users.select("Occupation").distinct().collect()]

# Dictionary za čuvanje tabela za svaku kombinaciju
tables = {}

# Prolazak kroz sve kombinacije polova i zanimanja
for gender in gender_list:
    for occupation in occupation_list:
        # Filtriranje za određeni pol i zanimanje
        filtered_df = top_3_movies_per_gender_occupation \
            .filter((col("Gender") == gender) & (col("Occupation") == occupation))\
            .select("Rank", "Title", "Broj_pregleda", "Prosecna_ocena", "Gender", "Occupation")

        # Čuvanje u dictionary pod nazivom tabele kao ključem
        table_name = f"top_3_movies_{gender}_{occupation}"
        tables[table_name] = filtered_df

        # Prikazivanje rezultata za svaku kombinaciju
        print(f"Top 3 filma za pol: {gender}, zanimanje: {occupation}")
        filtered_df.show(truncate = False)

5. Prikaz 3 najpopularnija filma po polu i zanimanju korisnika:
Top 3 filma za pol: F, zanimanje: librarian
+----+---------------------------+-------------+------------------+------+----------+
|Rank|Title                      |Broj_pregleda|Prosecna_ocena    |Gender|Occupation|
+----+---------------------------+-------------+------------------+------+----------+
|1   |English Patient, The (1996)|21           |3.9047619047619047|F     |librarian |
|2   |Star Wars (1977)           |17           |4.235294117647059 |F     |librarian |
|3   |Full Monty, The (1997)     |17           |4.0               |F     |librarian |
+----+---------------------------+-------------+------------------+------+----------+

Top 3 filma za pol: F, zanimanje: retired
+----+-------------------------------------------------------------------------------+-------------+--------------+------+----------+
|Rank|Title                                                                          |Broj_pregleda|Prosecna_ocen

**3 NAJPOPULARNIJA FILMA U SVAKOM ZANRU**

In [ ]:
# 6. Prikaz 3 najpopularnija filma u svakom žanru
print("6. Prikaz 3 najpopularnija filma u svakom žanru")
for genre in movies.columns[5:]:
    print(f"{genre}:")
    popular_movies_genre = ratings.join(movies, ratings.Movie_id == movies.Id).filter(col(genre) == 1).groupBy("Title").agg(count("Rating").alias("Broj_pregleda"), avg("Rating").alias("Prosecna_ocena"))
    popular_movies_genre.orderBy(col("Broj_pregleda").desc()).show(3, truncate = False)


6. Prikaz 3 najpopularnija filma u svakom žanru
Unknown:
+-------------------+-------------+------------------+
|Title              |Broj_pregleda|Prosecna_ocena    |
+-------------------+-------------+------------------+
|unknown            |9            |3.4444444444444446|
|Good Morning (1971)|1            |1.0               |
+-------------------+-------------+------------------+

Action:
+-------------------------+-------------+------------------+
|Title                    |Broj_pregleda|Prosecna_ocena    |
+-------------------------+-------------+------------------+
|Star Wars (1977)         |583          |4.3584905660377355|
|Return of the Jedi (1983)|507          |4.007889546351085 |
|Air Force One (1997)     |431          |3.6310904872389793|
+-------------------------+-------------+------------------+
only showing top 3 rows

Adventure:
+------------------------------+-------------+------------------+
|Title                         |Broj_pregleda|Prosecna_ocena    |
+--------

**KREIRANJE ATRIBUTA STAROSNA_GRUPA I PRIKAZ 5 NAJPOPULARNIJIH FILMOVA ZA SVAKU GRUPU**

In [ ]:
# 7. Kreiranje atributa kategorički starosna_grupa i prikaz 5 najpopularnijih filmova za svaku grupu
from pyspark.sql import Window
from pyspark.sql.functions import row_number, count, avg

# Kreiranje starosnih grupa
users = users.withColumn("Age", col("Age").cast("integer"))
age_groups = users.withColumn("Starosna_grupa", expr(
    "CASE WHEN Age <= 18 THEN 'Mlađi od 18' " +
    "WHEN Age BETWEEN 19 AND 35 THEN '19-35' " +
    "WHEN Age BETWEEN 36 AND 50 THEN '36-50' " +
    "ELSE 'Stariji od 50' END"
))

# Spajanje ocena i filmova sa starosnim grupama korisnika
ratings_movies = ratings.join(movies, movies.Id == ratings.Movie_id)
popular_movies_age_group = ratings_movies.join(age_groups, age_groups.Id == ratings_movies.User_id)

# Agregacija: broj pregleda i prosečna ocena po starosnoj grupi i filmu
popular_movies_age_group = popular_movies_age_group.groupBy("Starosna_grupa", "Title") \
    .agg(count("Rating").alias("Broj_pregleda"), avg("Rating").alias("Prosecna_ocena"))

# Kreiranje prozora za rangiranje filmova unutar svake starosne grupe
window = Window.partitionBy("Starosna_grupa").orderBy(col("Broj_pregleda").desc())

# Dodavanje rednog broja unutar svake grupe (za rangiranje)
popular_movies_age_group = popular_movies_age_group.withColumn("Rank", row_number().over(window))

# Filtriranje top 5 filmova za svaku starosnu grupu
top_5_movies_age_group = popular_movies_age_group.filter(col("Rank") <= 5)

# Prikaz top 5 filmova za svaku starosnu grupu
print("7. Kreiranje atributa kategorički starosna_grupa i prikaz 5 najpopularnijih filmova za svaku grupu:")

print("Top 5 filmova za starosnu grupu: Mlađi od 18")
top_5_movies_age_group.filter(col("Starosna_grupa") == "Mlađi od 18") \
    .select("Title", "Broj_pregleda", "Prosecna_ocena") \
    .show(truncate=False)

print("Top 5 filmova za starosnu grupu: 19-35")
top_5_movies_age_group.filter(col("Starosna_grupa") == "19-35") \
    .select("Title", "Broj_pregleda", "Prosecna_ocena") \
    .show(truncate=False)

print("Top 5 filmova za starosnu grupu: 36-50")
top_5_movies_age_group.filter(col("Starosna_grupa") == "36-50") \
    .select("Title", "Broj_pregleda", "Prosecna_ocena") \
    .show(truncate=False)

print("Top 5 filmova za starosnu grupu: Stariji od 50")
top_5_movies_age_group.filter(col("Starosna_grupa") == "Stariji od 50") \
    .select("Title", "Broj_pregleda", "Prosecna_ocena") \
    .show(truncate=False)


7. Kreiranje atributa kategorički starosna_grupa i prikaz 5 najpopularnijih filmova za svaku grupu:
Top 5 filmova za starosnu grupu: Mlađi od 18
+-------------------------+-------------+------------------+
|Title                    |Broj_pregleda|Prosecna_ocena    |
+-------------------------+-------------+------------------+
|Scream (1996)            |44           |3.8636363636363638|
|Contact (1997)           |33           |3.9696969696969697|
|Liar Liar (1997)         |32           |3.25              |
|Star Wars (1977)         |32           |4.625             |
|Return of the Jedi (1983)|29           |4.551724137931035 |
+-------------------------+-------------+------------------+

Top 5 filmova za starosnu grupu: 19-35
+-------------------------+-------------+------------------+
|Title                    |Broj_pregleda|Prosecna_ocena    |
+-------------------------+-------------+------------------+
|Star Wars (1977)         |353          |4.390934844192635 |
|Return of the Jedi (1

**UKUPNA PROSECNA OCENA SVIH FILMOVA**

In [ ]:
# 8. Prikaz ukupne prosečne ocene svih filmova
ukupna_prosecna_ocena = ratings.agg(avg("Rating").alias("Prosecna_ocena_svih_filmova"))
print("8. Prikaz ukupne prosečne ocene svih filmova:")
ukupna_prosecna_ocena.show()

8. Prikaz ukupne prosečne ocene svih filmova:
+---------------------------+
|Prosecna_ocena_svih_filmova|
+---------------------------+
|                    3.52986|
+---------------------------+



# **2. DEO: SISTEM PREPORUKE BAZIRAN NA SARDZAJU**

**EKSTRAHOVANJE KOLONE 'Year' (godina kada je film snimljen)**

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

# Ekstrakcija godine iz naziva filma
from pyspark.sql.functions import regexp_extract
movies = movies.withColumn('Year', regexp_extract('Title', r'\((\d{4})\)', 1).cast('int'))

print("Filmovi sa novom kolonom 'Year':")
movies.show(truncate=False)

Filmovi sa novom kolonom 'Year':
+---+----------------------------------------------------+------------+------------------+-------------------------------------------------------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+----+
|Id |Title                                               |Release date|Video release date|IMDb URL                                                           |Unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|Year|
+---+----------------------------------------------------+------------+------------------+-------------------------------------------------------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+----+
|1  |Toy Stor

**IMPORTOVANJE TRENING I TEST SKUPA I SREDJIVANJE PODATAKA**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor, LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

# Učitavanje u1_base i u1_test
u1_base = spark.read.option("delimiter", "\t").csv(f"{data_path}/u1.base", inferSchema=True, header=False).toDF("User_id", "Movie_id", "Rating", "Timestamp")
u1_test = spark.read.option("delimiter", "\t").csv(f"{data_path}/u1.test", inferSchema=True, header=False).toDF("User_id", "Movie_id", "Rating", "Timestamp")

# Spajanje sa ratings i users
train_data = u1_base.join(movies, u1_base.Movie_id == movies.Id, "inner")\
                    .join(users, u1_base.User_id == users.Id, "inner")

test_data = u1_test.join(movies, u1_test.Movie_id == movies.Id, "inner")\
                   .join(users, u1_test.User_id == users.Id, "inner")

# Konvertovanje kolone 'Gender' u numeričke vrednosti
gender_indexer = StringIndexer(inputCol="Gender", outputCol="Gender_Index")

# Fit na trening podacima
gender_indexer_model = gender_indexer.fit(train_data)
train_data = gender_indexer_model.transform(train_data)
test_data = gender_indexer_model.transform(test_data)
# Konvertovanje 'Occupation' u numeričke vrednosti
occupation_indexer = StringIndexer(inputCol="Occupation", outputCol="Occupation_Index")

# Fit i transformiši na trening podacima
occupation_indexer_model = occupation_indexer.fit(train_data)
train_data = occupation_indexer_model.transform(train_data)
test_data = occupation_indexer_model.transform(test_data)

# Konvertovanje 'Age', 'Rating' u int
train_data = train_data.withColumn("Age", col("Age").cast("int"))
train_data = train_data.withColumn("Rating", col("Rating").cast("int"))

test_data = test_data.withColumn("Age", col("Age").cast("int"))
test_data = test_data.withColumn("Rating", col("Rating").cast("int"))
from pyspark.sql.functions import col

# Proveri da li postoje null vrednosti
train_data_nulls = train_data.filter(
    col("Age").isNull() | col("Gender_Index").isNull() | col("Occupation_Index").isNull() | col("Year").isNull()
)

print("Broj redova sa null vrednostima u trening podacima:", train_data_nulls.count())

# Ukloni redove sa null vrednostima
train_data = train_data.na.drop(subset=["Age", "Gender_Index", "Occupation_Index", "Year"])

# Proveri i ukloni null vrednosti u test podacima
test_data_nulls = test_data.filter(
    col("Age").isNull() | col("Gender_Index").isNull() | col("Occupation_Index").isNull() | col("Year").isNull()
)

print("Broj redova sa null vrednostima u test podacima:", test_data_nulls.count())

test_data = test_data.na.drop(subset=["Age", "Gender_Index", "Occupation_Index", "Year"])
test_data = test_data
print("Trening podaci:")
train_data.show()
print("Test podaci:")
test_data.show()

Broj redova sa null vrednostima u trening podacima: 3
Broj redova sa null vrednostima u test podacima: 6
Trening podaci:
+-------+--------+------+---------+---+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+----+---+---+------+----------+--------+------------+----------------+
|User_id|Movie_id|Rating|Timestamp| Id|               Title|Release date|Video release date|            IMDb URL|Unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|Year| Id|Age|Gender|Occupation|Zip code|Gender_Index|Occupation_Index|
+-------+--------+------+---------+---+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+-----

**KREIRANJE VEKTORA ZA ULAZNE PODATKE**

In [ ]:
# Kreiranje input atributa i izlazne varijable
assembler = VectorAssembler(
    inputCols=['Age', 'Unknown',
 'Action',
 'Adventure',
 'Animation',
 'Children\'s',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western',
 'Year',
 'Gender_Index',
 'Occupation_Index',],
    outputCol="features")
train_data = assembler.transform(train_data).select("Movie_id", "features", "Rating")
test_data = assembler.transform(test_data).select("Movie_id", "features", "Rating")

**KONACNI PODACI ZA TRENING I TEST**

In [ ]:
print("Trening podaci final:")
train_data.show(truncate=False)
print("Test podaci final:")
test_data.show(truncate=False)

Trening podaci final:
+--------+----------------------------------------------------------------+------+
|Movie_id|features                                                        |Rating|
+--------+----------------------------------------------------------------+------+
|1       |(23,[0,4,5,6,20,22],[24.0,1.0,1.0,1.0,1995.0,8.0])              |5     |
|2       |(23,[0,2,3,17,20,22],[24.0,1.0,1.0,1.0,1995.0,8.0])             |3     |
|3       |(23,[0,17,20,22],[24.0,1.0,1995.0,8.0])                         |4     |
|4       |(23,[0,2,6,9,20,22],[24.0,1.0,1.0,1.0,1995.0,8.0])              |3     |
|5       |(23,[0,7,9,17,20,22],[24.0,1.0,1.0,1.0,1995.0,8.0])             |3     |
|7       |(23,[0,9,16,20,22],[24.0,1.0,1.0,1995.0,8.0])                   |4     |
|8       |(23,[0,5,6,9,20,22],[24.0,1.0,1.0,1.0,1995.0,8.0])              |1     |
|9       |(23,[0,9,20,22],[24.0,1.0,1995.0,8.0])                          |5     |
|11      |(23,[0,7,17,20,22],[24.0,1.0,1.0,1995.0,8.0])          

**KREIRANJE I TRENIRANJE RANDOM FOREST MODELA I LINEARNE REGRESIJE**

In [ ]:
# Kreiranje i treniranje Random Forest regresije
rf = RandomForestRegressor(featuresCol="features", labelCol="Rating")
rf_evaluator = RegressionEvaluator(labelCol="Rating", predictionCol="prediction")

print("Training random forest...")
rf_model = rf.fit(train_data)

print("Training linear regression...")
# Kreiranje i treniranje Linearne regresije
lr = LinearRegression(featuresCol="features", labelCol="Rating")
lr_model = lr.fit(train_data)
test_results = lr_model.evaluate(test_data)

# Predikcije za test skup
rf_predictions = rf_model.transform(test_data)
lr_predictions = lr_model.transform(test_data)

# Evaluacija modela
rf_rmse = rf_evaluator.evaluate(rf_predictions, {rf_evaluator.metricName: "rmse"})
rf_mse = rf_evaluator.evaluate(rf_predictions, {rf_evaluator.metricName: "mse"})
rf_r2 = rf_evaluator.evaluate(rf_predictions, {rf_evaluator.metricName: "r2"})
print("- RANDOM FOREST: ")
print("RMSE:", rf_rmse)
print("MSE:", rf_mse)
print("R2:", rf_r2)
rf_predictions.show()
print()
print("- LINEARNA REGRESIJA:")
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))
lr_predictions.show()

Training random forest...
Training linear regression...
- RANDOM FOREST: 
RMSE: 1.0962309096367422
MSE: 1.2017222072429992
R2: 0.09722484897853612
+--------+--------------------+------+------------------+
|Movie_id|            features|Rating|        prediction|
+--------+--------------------+------+------------------+
|       6|(23,[0,9,20,22],[...|     5| 3.593376781165363|
|      10|(23,[0,9,18,20,22...|     3|3.6495586971402667|
|      12|(23,[0,7,17,20,22...|     5| 3.434703499444609|
|      14|(23,[0,9,15,20,22...|     5|3.5913519384289203|
|      17|(23,[0,2,6,7,12,1...|     3|3.2914756514091956|
|      20|(23,[0,9,15,20,22...|     4|3.5913519384289203|
|      23|(23,[0,9,17,20,22...|     4|  3.84105009153675|
|      24|(23,[0,2,3,7,20,2...|     3|3.2916706830109037|
|      27|(23,[0,2,20,22],[...|     2|  3.29058394620329|
|      31|(23,[0,9,17,18,20...|     3|3.6495586971402667|
|      33|(23,[0,2,15,17,20...|     4|3.3201604465271344|
|      36|(23,[0,9,15,20,22...|     2|3.5

**OPTIMIZACIJA HIPERPARAMETARA ZA MODELE**

In [ ]:
# OPTIMIZACIJA HIPERPARAMETARA

# RANDOM FOREST
rf = RandomForestRegressor(featuresCol="features", labelCol="Rating")
paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [5, 10, 20])
             .addGrid(rf.maxDepth, [3, 5, 9])
             .build())

evaluator = RegressionEvaluator(metricName="rmse",labelCol="Rating")

cv = CrossValidator(estimator=rf,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=3)

cv_model_rf = cv.fit(train_data)
bestModelRF = cv_model_rf.bestModel

predictions_rf = bestModelRF.transform(test_data)

#Evaluacija modela
rmse = evaluator.evaluate(predictions_rf)
print("- RANDOM FOREST -")
print("RMSE: %g" % rmse)
print("Optimalni hiperparametri:")
print("numTrees: ", bestModelRF.getNumTrees)
print("maxDepth: ", bestModelRF.getMaxDepth())
print()

#LINEARNA REGRESIJA
lr = LinearRegression(featuresCol="features", labelCol="Rating", maxIter=10)

# Define a grid of hyperparameters to search over
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()


evaluator = RegressionEvaluator(metricName="rmse",labelCol="Rating")

#Krosvalidacija
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)


cv_model_lr = crossval.fit(train_data)


predictions_lr = cv_model_lr.transform(test_data)

# Evaluacija modela
rmse = evaluator.evaluate(predictions_lr)
print("- LINEARNA REGRESIJA -")
print("RMSE: %g" % rmse)

# Optimalni hiperparametri
print("Optimalni hiperparametri:")
print("regParam: ", cv_model_lr.bestModel._java_obj.getRegParam())
print("elasticNetParam: ", cv_model_lr.bestModel._java_obj.getElasticNetParam())

- RANDOM FOREST -
RMSE: 1.06887
Optimalni hiperparametri:
numTrees:  20
maxDepth:  9

- LINEARNA REGRESIJA -
RMSE: 1.11425
Optimalni hiperparametri:
regParam:  0.01
elasticNetParam:  0.0


**PROCEDURA ZA PREPORUKU**

In [ ]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

def predict_ranking_for_user(age, gender, occupation, top_movies, model):
    # Prvo, kreiramo dataframe sa 50 najpopularnijih filmova
    # Pretpostavimo da `top_movies` sadrži atribute filmova koje treba koristiti za preporuke
    movies_df = top_movies.select('Id', 'Title', 'Year', 'Unknown', 'Action', 'Adventure', 'Animation', 'Children\'s',
                                  'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
                                  'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western')

    # Kreiraj dataframe sa korisničkim informacijama
    user_row = Row(Age=age, Gender=gender, Occupation=occupation)
    user_df = spark.createDataFrame([user_row])

    # Konvertovanje kolona 'Gender' i 'Occupation' u numeričke vrednosti, koristeći prethodno kreirane StringIndexer modele
    user_df = gender_indexer_model.transform(user_df)
    user_df = occupation_indexer_model.transform(user_df)

    # Dodajemo korisničke atribute filmovima (cross join sa top_movies)
    user_movies = movies_df.crossJoin(user_df.select('Age', 'Gender_Index', 'Occupation_Index'))

    # Kreiranje feature vektora za predikciju
    features_assembler = VectorAssembler(
        inputCols=['Age', 'Unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime',
                   'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
                   'Sci-Fi', 'Thriller', 'War', 'Western', 'Year', 'Gender_Index', 'Occupation_Index'],
        outputCol="features"
    )

    # Transformišemo user_movies u dataset sa feature kolonom
    user_movies = features_assembler.transform(user_movies).select('Id', 'Title', 'features')

    # Predikcija ocena za 50 filmova pomoću modela sa najboljim performansama (RF sa optimizovanim hiperparametrima jer ima najbolji RMSE)
    predictions = model.transform(user_movies).select('Id', 'Title', 'features', 'prediction')

    # Dodavanje rangiranja
    window_spec = Window.orderBy(col('prediction').desc())
    ranked_predictions = predictions.withColumn('Rank', row_number().over(window_spec))

    # Sortiranje filmova po rangiranju
    sorted_movies = ranked_predictions.orderBy(col('Rank'))

    # Prikaz rangiranih filmova
    sorted_movies.select('Rank', 'Title', 'features', 'prediction').show(50, truncate=False)

    return sorted_movies

# Primer poziva funkcije
popular_movies = ratings.groupBy("Movie_id").agg(count("Rating").alias("Broj_pregleda"), avg("Rating").alias("Prosecna_ocena"))
popular_movies = popular_movies.join(movies, popular_movies.Movie_id == movies.Id).orderBy(col("Broj_pregleda").desc())
top_50_movies = popular_movies.orderBy('Broj_pregleda', ascending=False).limit(50)

print(f"Preporučeni filmovi za korisnika: Pol=F, Godine=24, Zanimanje=programmer")
sorted_movies = predict_ranking_for_user(24, 'F', 'programmer', top_50_movies, bestModelRF)


Preporučeni filmovi za korisnika: Pol=F, Godine=24, Zanimanje=programmer
+----+--------------------------------------------+------------------------------------------------------------------------------+------------------+
|Rank|Title                                       |features                                                                      |prediction        |
+----+--------------------------------------------+------------------------------------------------------------------------------+------------------+
|1   |Star Wars (1977)                            |(23,[0,2,3,15,16,18,20,21,22],[24.0,1.0,1.0,1.0,1.0,1.0,1977.0,1.0,4.0])      |4.327919683653465 |
|2   |Empire Strikes Back, The (1980)             |(23,[0,2,3,9,15,16,18,20,21,22],[24.0,1.0,1.0,1.0,1.0,1.0,1.0,1980.0,1.0,4.0])|4.272309873608231 |
|3   |Return of the Jedi (1983)                   |(23,[0,2,3,15,16,18,20,21,22],[24.0,1.0,1.0,1.0,1.0,1.0,1983.0,1.0,4.0])      |4.240897293300049 |
|4   |Silence of the Lambs,

# **3. DEO: SISTEM PREPORUKE BAZIRAN NA KOLABORATIVNOM FILTRIRANJU**

**KREIRANJE I TRENIRANJE ALS MODELA**

In [ ]:
################################ SISTEM PREPORUKE BAZIRAN NA KOLABORATIVNOM FILTRIRANJU ###############################

from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

# Učitavanje skupa podataka u1.base i u1.test
u1_base = spark.read.option("delimiter", "\t").csv(f"{data_path}/u1.base", inferSchema=True, header=False).toDF("User_id", "Movie_id", "Rating", "Timestamp")
trainingData = u1_base.withColumnRenamed("User_id", "userId").withColumnRenamed("Movie_id", "movieId")

u1_test = spark.read.option("delimiter", "\t").csv(f"{data_path}/u1.test", inferSchema=True, header=False).toDF("User_id", "Movie_id", "Rating", "Timestamp")
testData = u1_test.withColumnRenamed("User_id", "userId").withColumnRenamed("Movie_id", "movieId")

# Definišemo ALS model
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="Rating", coldStartStrategy="drop")
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating", predictionCol="prediction")

# Treniramo model
model = als.fit(trainingData)

# Evaluacija na test podacima
predictions = model.transform(testData)
rmse = evaluator.evaluate(predictions)
als_rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
als_mse = evaluator.evaluate(predictions, {evaluator.metricName: "mse"})
als_r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
print("- ALS MODEL -")
print(f"RMSE: {als_rmse}")
print(f"MSE: {als_mse}")
print(f"R2: {als_r2}")
predictions.show()
# Ako je RMSE visok, model može biti pod-treniran ili može biti potreban dodatni preprocesing.
# Ako je RMSE nizak, ali previše varira, može doći do pre-treniranja.

- ALS MODEL -
RMSE: 1.1276252238666244
MSE: 1.2715386455002553
R2: 0.04192795072501432
+------+-------+------+---------+----------+
|userId|movieId|Rating|Timestamp|prediction|
+------+-------+------+---------+----------+
|   148|      1|     4|877019411| 3.7106798|
|   148|      7|     5|877017054| 3.9092119|
|   148|     70|     5|877021271| 1.5155824|
|   148|     71|     5|877019251|  3.278763|
|   148|     78|     1|877399018| -1.363712|
|   148|     98|     3|877017714|  4.004921|
|   148|    114|     5|877016735| 5.3252406|
|   148|    116|     5|877398648| 2.1587446|
|   148|    140|     1|877019882| 1.3392434|
|   148|    163|     4|877021402| 5.0781527|
|   148|    169|     5|877020297|  3.676612|
|   148|    172|     5|877016513| 5.5424685|
|   148|    177|     2|877020715|  4.258935|
|   148|    185|     1|877398385| 4.0364356|
|   148|    204|     3|877016912| 3.9511635|
|   148|    214|     5|877019882|  4.942402|
|   148|    228|     4|877016514| 6.1955914|
|   148|    3

**OPTIMIZACIJA HIPERPARAMETARA ALS MODELA**

In [ ]:
# OPTIMIZACIJA HIPERPARAMETARA

alsParamGrid = ParamGridBuilder() \
    .addGrid(als.rank, [5, 10, 30]) \
    .addGrid(als.regParam, [0.01, 0.05, 0.1]) \
    .addGrid(als.maxIter, [5, 10, 15]) \
    .build()

als_evaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating", predictionCol="prediction")
als_crossval = CrossValidator(estimator=als,
                          estimatorParamMaps=alsParamGrid,
                          evaluator=als_evaluator,
                          numFolds=3)

cvModel = als_crossval.fit(trainingData)

# Najbolji model
bestModelAls = cvModel.bestModel

# Pravimo predikcije na trening skupu podataka
train_predictions = bestModelAls.transform(trainingData)
train_predictions.show(100)

# Računamo RMSE na trening skupu podataka
train_rmse = als_evaluator.evaluate(train_predictions)
print("RMSE na trening podacima:" + str(train_rmse))

predictions = bestModelAls.transform(testData)
predictions.show(100)
rmse = evaluator.evaluate(predictions)
print("RMSE:" + str(rmse))

+------+-------+------+---------+----------+
|userId|movieId|Rating|Timestamp|prediction|
+------+-------+------+---------+----------+
|   148|      8|     4|877020297| 3.9953666|
|   148|     50|     5|877016805|  4.818576|
|   148|     56|     5|877398212| 3.5743146|
|   148|     69|     5|877019101|  3.589503|
|   148|     89|     5|877398587|  4.395075|
|   148|    127|     1|877399351|  3.903019|
|   148|    132|     4|877020715| 3.8712323|
|   148|    133|     5|877019251| 3.3923125|
|   148|    135|     5|877016514| 4.0470924|
|   148|    151|     4|877400124| 3.6380043|
|   148|    164|     4|877398444|  3.646042|
|   148|    168|     5|877015900|  4.520213|
|   148|    173|     5|877017054| 4.6928782|
|   148|    174|     5|877015066| 4.4956083|
|   148|    175|     4|877016259| 4.0364103|
|   148|    181|     5|877399135| 4.4437704|
|   148|    189|     4|877019698|  4.168671|
|   148|    190|     2|877398586| 3.7931745|
|   148|    191|     1|877020715|  3.800658|
|   148|  

**ODVAJANJE FILMOVA KOJI SE NALAZE U U2.BASE I KOJI SE NALAZE U U2.BASE I U1.BASE**

In [ ]:
u2_base = spark.read.option("delimiter", "\t").csv(f"{data_path}/u2.base", inferSchema=True, header=False).toDF("User_id", "Movie_id", "Rating", "Timestamp")
u1_base = spark.read.option("delimiter", "\t").csv(f"{data_path}/u1.base", inferSchema=True, header=False).toDF("User_id", "Movie_id", "Rating", "Timestamp")
print("u2.base:")
u2_base.show()
print("u1.base:")
u1_base.show()
print()

u2_movie_ids = u2_base.select("Movie_id").distinct()
u1_movie_ids = u1_base.select("Movie_id").distinct()

movies_u2_only = u2_movie_ids.subtract(u1_movie_ids)
movies_u2_only = movies_u2_only.join(movies, movies_u2_only.Movie_id == movies.Id)
movies_u_12 = u2_movie_ids.intersect(u1_movie_ids)
movies_u_12 = movies_u_12.join(movies, movies_u_12.Movie_id == movies.Id)

# prikaz
print("Filmovi koji se ne nalaze u u1.base a nalaze se u u2.base:")
movies_u2_only.orderBy("Movie_id").select("Movie_id", "Title").show()
print("Filmovi koji se nalaze i u u1.base i u u2.base:")
movies_u_12.orderBy("Movie_id").select("Movie_id", "Title").show()

u2.base:
+-------+--------+------+---------+
|User_id|Movie_id|Rating|Timestamp|
+-------+--------+------+---------+
|      1|       3|     4|878542960|
|      1|       4|     3|876893119|
|      1|       5|     3|889751712|
|      1|       6|     5|887431973|
|      1|       7|     4|875071561|
|      1|      10|     3|875693118|
|      1|      11|     2|875072262|
|      1|      12|     5|878542960|
|      1|      13|     5|875071805|
|      1|      14|     5|874965706|
|      1|      15|     5|875071608|
|      1|      16|     5|878543541|
|      1|      17|     3|875073198|
|      1|      18|     4|887432020|
|      1|      19|     5|875071515|
|      1|      20|     4|887431883|
|      1|      23|     4|875072895|
|      1|      24|     3|875071713|
|      1|      25|     4|875071805|
|      1|      27|     2|876892946|
+-------+--------+------+---------+
only showing top 20 rows

u1.base:
+-------+--------+------+---------+
|User_id|Movie_id|Rating|Timestamp|
+-------+--------+--

**PROCEDURA ZA PODELU I PREDVIDJANJE**

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.sql.types import StructType, StructField, IntegerType, ArrayType, DoubleType
def divide_and_predict():
  # Grupisanje korisnika po broju ocena
  u2_user_count = u2_base.groupBy("User_id").agg(count("*").alias("u2_count"))
  u1_user_count = u1_base.groupBy("User_id").agg(count("*").alias("u1_count"))

  # Kreiramo users_with_grades i users_without_grades
  # Prvo kreiramo users_with_grades (korisnici sa 5 ili više ocena)
  users_with_grades = u2_user_count.join(u1_user_count, "User_id").filter(col("u2_count") >= 5)

  # Za users_without_grades koristi se left_anti da se identifikuju korisnici koji su samo u u2_user_count
  # Zatim se filtriraju oni koji imaju manje od 10 ocena u u2.base jer je najmanji broj ocena 9
  users_without_grades = u2_user_count.join(u1_user_count, "User_id", "left_anti").select("User_id", "u2_count").union(
      u2_user_count.join(u1_user_count, "User_id").filter(col("u2_count") < 10).select("User_id", "u2_count")
  )

  print("Korisnici iz u2.base koji se pojavljuju u u1.base i imaju minimum 5 ocena:")
  users_with_grades.show()
  print("Korisnici iz u2.base koji se ne pojavljuju u u1.base ili imaju manje od 10 ocena:")
  users_without_grades.show()

  users_with_grades_data = u2_base.join(users_with_grades, "User_id")

  users_with_grades_data = users_with_grades_data.withColumnRenamed("User_id", "userId").withColumnRenamed("Movie_id", "movieId")
  user_indexer = StringIndexer(inputCol="userId", outputCol="User_Index")
  user_indexer_model = user_indexer.fit(users_with_grades_data)
  users_with_grades_data = user_indexer_model.transform(users_with_grades_data)

  # ALS predikcije za korisnike sa ocenama
  als_predictions = bestModel.transform(users_with_grades_data.drop("u2_count", "u1_count", "User_Index"))
  print("ALS predikcije za korisnike sa ocenama:")
  als_predictions.show()
  users_without_grades_data = users_without_grades.join(users, users_without_grades.User_id == users.Id)

  print("Predvidjanje za korisnike koji nisu u u1.base ili imaju manje od 10 ocena - na osnovu RandomForest modela sa optimizovanim hiperparametrima:")
  print()
  for row in users_without_grades_data.collect():
    age = row['Age']
    gender = row['Gender']
    occupation = row['Occupation']

    # Predviđanje na osnovu random forest modela
    print(f"Korisnik: Id={row['User_id']}, Pol={gender}, Godine={age}, Zanimanje={occupation}")
    sorted_movies = predict_ranking_for_user(age, gender, occupation, top_50_movies, bestModelRF).select("Rank", "Title", "features", "prediction")
  return als_predictions


**PRONALAZENJE KORISNIKA SA NAJVECIM BROJEM OCENA I PRIKAZ REZULTATA**

In [ ]:
# Primena procedure i prikaz preporuka
predictions = divide_and_predict()

# uzima se id najaktivnijeg korisnika (korisnik sa najvise ocena)
most_active_user = u2_base.groupBy("User_id").count().orderBy(col("count").desc()).first()["User_id"]
print(f"Predikcije za korisnika koji je imao najvise ocena u u2.base: Id = {most_active_user}")
predictions_for_the_most_active_user = predictions.filter(col("userId") == most_active_user).orderBy(col("prediction").desc()).show()

Korisnici iz u2.base koji se pojavljuju u u1.base i imaju minimum 5 ocena:
+-------+--------+--------+
|User_id|u2_count|u1_count|
+-------+--------+--------+
|    148|      55|      33|
|    463|      74|     133|
|    471|      17|      31|
|    496|      75|     129|
|    833|     267|     267|
|    243|      61|      45|
|    392|      76|      81|
|    540|      49|      63|
|    623|      39|      45|
|    737|      33|      33|
|    858|      21|      21|
|    897|     185|     185|
|     31|      26|      23|
|    516|      13|      21|
|     85|     211|     159|
|    137|      32|      24|
|    251|      52|      44|
|    451|      59|      91|
|    580|      37|      47|
|    808|      23|      23|
+-------+--------+--------+
only showing top 20 rows

Korisnici iz u2.base koji se ne pojavljuju u u1.base ili imaju manje od 10 ocena:
+-------+--------+
|User_id|u2_count|
+-------+--------+
|    512|       9|
|    511|       9|
+-------+--------+

ALS predikcije za korisnike sa